<a href="https://colab.research.google.com/github/altair08/FYP/blob/main/predict_LSTM_with_subword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

In [28]:
# Load the saved model
model = load_model('/content/drive/MyDrive/LSTM_with_bidirectional')

In [43]:
df1 = pd.read_csv('/content/drive/MyDrive/Dats/reddit data/preprocessed_data.csv')
sentiment = df1['preprocessed_text'].values


In [44]:
df1

,Title,Author,ID,preprocessed_text
0,UkrainianConflict Discussion Megathread,humanlikecorvus,y7gz80,ukrainianconflict discussion megathread
1,Zelenskyy survives over 12 assassination attem...,Far-Childhood9338,10e17wq,zelenskyy survive assassination attempt since ...
2,In the first round of presidential elections i...,RevealDisinfo,10digs3,first round presidential election czech republ...
3,"A further 20,000 Ukrainian recruits will be tr...",tedwja,10dv085,ukrainian recruit train uk year
4,"Zelensky: ""Tanks, APCs and artillery are exact...",zizp,10duei9,zelensky tank apc artillery exactly ukraine ne...
...,...,...,...,...
971,Hundreds of US military vehicles arrive in Dut...,Standard_Spaniard,109fbyj,hundred u military vehicle arrive dutch port s...
972,BREAKING: Poland will deliver a company of Leo...,rulepanic,1096adh,break poland deliver company leopard tank ukra...
973,Russian airline aircraft suffer massive breakd...,Breech_Loader,109envz,russian airline aircraft suffer massive breakd...
974,"The Russian Federation declared that it ""has t...",RevealDisinfo,1095iye,russian federation declare right launch nuclea...


In [56]:
# Preprocess the new dataset df1
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(list(sentiment))  # Assuming sentiment is the training dataset
predict_data = df1['preprocessed_text'].values
test_seq = tokenizer.texts_to_sequences(predict_data)
test_pad = pad_sequences(test_seq, maxlen=model.input_shape[1])


In [60]:
# Make predictions on the new dataset
y_test_with_subword = model.predict([test_pad], batch_size=32, verbose=1)

# Apply threshold and convert to 0 or 1
threshold = 0.01
y_test_with_subword[y_test_with_subword >= threshold] = 1
y_test_with_subword[y_test_with_subword < threshold] = 0
y_test_with_subword = y_test_with_subword.astype(int)

# Create a DataFrame with the predicted labels
y_list = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
predict_labels_df_with_subword = pd.DataFrame(y_test_with_subword, columns=y_list)

# Create a DataFrame with the original data from df1
predict_data_df = pd.DataFrame(predict_data, columns=["Title"])

# Combine the original DataFrame with the predicted labels
predict_df_with_subword = pd.concat([predict_data_df, predict_labels_df_with_subword], axis=1)

# Save the DataFrame to a CSV file
predict_df_with_subword.to_csv('/content/drive/MyDrive/predict_lstm_with_subword_reddit.csv', index=False)

# Print the predicted DataFrame
predict_df_with_subword


31/31 [==============================] - 1s 30ms/step


,Title,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,ukrainianconflict discussion megathread,1,0,1,1,1,0
1,zelenskyy survive assassination attempt since ...,0,0,0,0,0,0
2,first round presidential election czech republ...,1,0,0,0,1,0
3,ukrainian recruit train uk year,0,0,0,0,0,0
4,zelensky tank apc artillery exactly ukraine ne...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
971,hundred u military vehicle arrive dutch port s...,0,0,0,0,0,0
972,break poland deliver company leopard tank ukra...,0,0,0,0,0,0
973,russian airline aircraft suffer massive breakd...,1,0,0,0,0,0
974,russian federation declare right launch nuclea...,0,0,0,0,0,0
